## Домашнее задание

Вам нужно написать следующий пайплайн:
1. Получить эмбединги пользователей с помощью SVD.
2. Подать их в нейронную сеть, которую вы делали на практическом занятии.
Она принимает на вход пользователей и эмбединг айтомов, далее объединяет их в два полносвязанных слоя. На последнем слое выдаёт прогноз с функцией активации сигмоид.
3. Обучить сеть.


In [1]:
import tensorflow as tf
from keras.regularizers import l1, l2
from keras.models import Model, load_model, save_model, clone_model
from keras.layers import Embedding, Input, Dense, Reshape, Flatten, Dropout, Concatenate, Multiply
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
import warnings
warnings.filterwarnings("ignore")
from surprise import SVD  
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import SVD  
data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [2]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [3]:
from surprise.model_selection import cross_validate
for i in [1,2,3,5,7,10,50,70]:
    algo = SVD(n_factors=i,random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'-factors:',np.mean(cv['test_rmse']))

1-factors: 0.9433492611629224
2-factors: 0.9434062255128935
3-factors: 0.9436283884407395
5-factors: 0.9440460159093239
7-factors: 0.9438463194688421
10-factors: 0.9434708525192903
50-factors: 0.9436089265351626
70-factors: 0.9427779100472841


In [4]:
algo = SVD(n_factors=2,random_state=999)
algo.fit(trainset).test(testset)

[Prediction(uid='196', iid='302', r_ui=3.52986, est=4.220482524653445, details={'was_impossible': False}),
 Prediction(uid='196', iid='377', r_ui=3.52986, est=2.5635413201470403, details={'was_impossible': False}),
 Prediction(uid='196', iid='51', r_ui=3.52986, est=3.4573887826819667, details={'was_impossible': False}),
 Prediction(uid='196', iid='346', r_ui=3.52986, est=3.747024369274875, details={'was_impossible': False}),
 Prediction(uid='196', iid='474', r_ui=3.52986, est=4.335488660790024, details={'was_impossible': False}),
 Prediction(uid='196', iid='265', r_ui=3.52986, est=3.9049577174883807, details={'was_impossible': False}),
 Prediction(uid='196', iid='465', r_ui=3.52986, est=3.5422432147371046, details={'was_impossible': False}),
 Prediction(uid='196', iid='451', r_ui=3.52986, est=3.308483403392578, details={'was_impossible': False}),
 Prediction(uid='196', iid='86', r_ui=3.52986, est=3.945485786838208, details={'was_impossible': False}),
 Prediction(uid='196', iid='1014', 

In [5]:
algo.fit(trainset)

In [6]:
algo.qi

array([[-0.1928537 , -0.21347833],
       [ 0.35794515, -0.05124416],
       [-0.07722728, -0.07372941],
       ...,
       [-0.02716054, -0.1669574 ],
       [ 0.08794653,  0.01216776],
       [ 0.14119523,  0.1244365 ]])

In [7]:
df_pu = pd.DataFrame({'pu': [i for i in algo.pu], 'user': range(1, len(algo.pu)+1)}) 
#Датафрейм из эмбендингов пользователей
df_pu.head(5)

,pu,user
0,"[0.09891286210444199, 0.09727714653816635]",1
1,"[0.0011284409446351653, -0.12365285881685911]",2
2,"[-0.015195870254740557, 0.09640138541422033]",3
3,"[0.29026351226630437, 0.02941152744366906]",4
4,"[0.0382555833343881, 0.08383319846231022]",5


In [8]:
df_qi = pd.DataFrame({'qi': [i for i in algo.qi], 'item': range(1, len(algo.qi)+1)})
df_qi.head(5)

,qi,item
0,"[-0.19285370429312113, -0.21347833268374877]",1
1,"[0.35794515406654304, -0.05124416064850632]",2
2,"[-0.07722727832650503, -0.07372941408193771]",3
3,"[0.01614217929534281, -0.05767413898999959]",4
4,"[0.41046269822011233, 0.25517057953562666]",5


In [9]:
df_pu[df_pu["user"]==196]

,pu,user
195,"[0.11878943957994438, 0.05586335409901004]",196


In [10]:
df_qi[df_qi["item"]==242]

,qi,item
241,"[0.06231603546628062, 0.1595322394399691]",242


In [11]:
df['user'] = df.user.astype(int)
df['item'] = df.item.astype(int)

df = df.merge(df_pu, on='user') #объединяем основной датафрейм с эмедингами пользователей
df = df.merge(df_qi, on='item') #и айтемов
df.head(5)

,user,item,rating,timestamp,pu,qi
0,196,242,3.0,881250949,"[0.11878943957994438, 0.05586335409901004]","[0.06231603546628062, 0.1595322394399691]"
1,305,242,5.0,886307828,"[0.18851380423852224, 0.14777265340651216]","[0.06231603546628062, 0.1595322394399691]"
2,6,242,4.0,883268170,"[-0.08905044520747311, -0.015349350675485525]","[0.06231603546628062, 0.1595322394399691]"
3,234,242,4.0,891033261,"[-0.08275714693313996, -0.23542206053916614]","[0.06231603546628062, 0.1595322394399691]"
4,63,242,3.0,875747190,"[-0.005148684292168395, 0.020823577321531714]","[0.06231603546628062, 0.1595322394399691]"


In [12]:
df['X'] = df.apply(lambda x: np.concatenate((x['pu'], x['qi'])), axis=1) #объединяем эмбединги
df['y'] = df['rating'].apply(lambda x: 1 if x > 3 else 0) #оценка больше 3 — релевантная рекомендация
df.head(5)  

,user,item,rating,timestamp,pu,qi,X,y
0,196,242,3.0,881250949,"[0.11878943957994438, 0.05586335409901004]","[0.06231603546628062, 0.1595322394399691]","[0.11878943957994438, 0.05586335409901004, 0.0...",0
1,305,242,5.0,886307828,"[0.18851380423852224, 0.14777265340651216]","[0.06231603546628062, 0.1595322394399691]","[0.18851380423852224, 0.14777265340651216, 0.0...",1
2,6,242,4.0,883268170,"[-0.08905044520747311, -0.015349350675485525]","[0.06231603546628062, 0.1595322394399691]","[-0.08905044520747311, -0.015349350675485525, ...",1
3,234,242,4.0,891033261,"[-0.08275714693313996, -0.23542206053916614]","[0.06231603546628062, 0.1595322394399691]","[-0.08275714693313996, -0.23542206053916614, 0...",1
4,63,242,3.0,875747190,"[-0.005148684292168395, 0.020823577321531714]","[0.06231603546628062, 0.1595322394399691]","[-0.005148684292168395, 0.020823577321531714, ...",0


In [13]:
df.X[0]

array([0.11878944, 0.05586335, 0.06231604, 0.15953224])

In [14]:
X, y = df['X'].values, df['y'].values
X

array([array([0.11878944, 0.05586335, 0.06231604, 0.15953224]),
       array([0.1885138 , 0.14777265, 0.06231604, 0.15953224]),
       array([-0.08905045, -0.01534935,  0.06231604,  0.15953224]), ...,
       array([-0.02402643,  0.40336113, -0.02716054, -0.1669574 ]),
       array([-0.40190093, -0.02993142,  0.08794653,  0.01216776]),
       array([-0.13443814, -0.01644649,  0.14119523,  0.1244365 ])],
      dtype=object)

In [15]:
X = [list(i) for i in X]

In [16]:
X = np.array(X)
X

array([[ 0.11878944,  0.05586335,  0.06231604,  0.15953224],
       [ 0.1885138 ,  0.14777265,  0.06231604,  0.15953224],
       [-0.08905045, -0.01534935,  0.06231604,  0.15953224],
       ...,
       [-0.02402643,  0.40336113, -0.02716054, -0.1669574 ],
       [-0.40190093, -0.02993142,  0.08794653,  0.01216776],
       [-0.13443814, -0.01644649,  0.14119523,  0.1244365 ]])

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(125, input_shape=(4,), activation='relu'),
    tf.keras.layers.Dense(75, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
loss = tf.keras.losses.binary_crossentropy
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [19]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 125)               625       
                                                                 
 dense_1 (Dense)             (None, 75)                9450      
                                                                 
 dense_2 (Dense)             (None, 25)                1900      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 12,001
Trainable params: 12,001
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model.fit(x_train, 
          y_train, 
          validation_data=(x_test, y_test),
          epochs = 500,
          batch_size = 32)

Epoch 1/500
2188/2188 [==============================] - 8s 3ms/step - loss: 0.6866 - accuracy: 0.5534 - val_loss: 0.6862 - val_accuracy: 0.5532
Epoch 2/500
2188/2188 [==============================] - 7s 3ms/step - loss: 0.6859 - accuracy: 0.5541 - val_loss: 0.6859 - val_accuracy: 0.5538
Epoch 3/500
2188/2188 [==============================] - 6s 3ms/step - loss: 0.6852 - accuracy: 0.5554 - val_loss: 0.6851 - val_accuracy: 0.5550
Epoch 4/500
2188/2188 [==============================] - 7s 3ms/step - loss: 0.6844 - accuracy: 0.5565 - val_loss: 0.6846 - val_accuracy: 0.5564
Epoch 5/500
2188/2188 [==============================] - 6s 3ms/step - loss: 0.6835 - accuracy: 0.5584 - val_loss: 0.6837 - val_accuracy: 0.5569
Epoch 6/500
2188/2188 [==============================] - 7s 3ms/step - loss: 0.6828 - accuracy: 0.5605 - val_loss: 0.6831 - val_accuracy: 0.5594
Epoch 7/500
2188/2188 [==============================] - 7s 3ms/step - loss: 0.6821 - accuracy: 0.5606 - val_loss: 0.6819 - val_ac

In [21]:
## Проверка метрик на тестовом наборе данных:
model.evaluate(x_test, y_test)

938/938 [==============================] - 2s 2ms/step - loss: 0.7062 - accuracy: 0.5894


[0.7062393426895142, 0.5894333124160767]

In [28]:
y_predict = model.predict(x_test)
y_predict

938/938 [==============================] - 2s 2ms/step


array([[0.59371793],
       [0.63250774],
       [0.5658066 ],
       ...,
       [0.5447753 ],
       [0.6029874 ],
       [0.56355935]], dtype=float32)